In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [ ]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/amrragababdelaziz/dermnet-and-skin-disease-9-classes')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 3.51G/3.51G [02:58<00:00, 21.1MB/s]


In [ ]:
train ='/content/dermnet-and-skin-disease-9-classes/Merged Dataset/Train'
validation_dir = '/content/dermnet-and-skin-disease-9-classes/Merged Dataset/test'

In [ ]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [ ]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

9 Classes
26642 Train images
2389 Test images


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
img_width,img_height =224,224
input_shape=(img_width,img_height,3)
batch_size =32
train_generator =train_datagen.flow_from_directory(train,
                                target_size=(img_width,img_height),batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(validation_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 26642 images belonging to 9 classes.
Found 2389 images belonging to 9 classes.


In [ ]:
from tensorflow import keras
from keras import layers
model = keras.Sequential([
    layers.Input((224, 224, 3)),
    
    layers.Conv2D(16, 3, activation="relu"),
    layers.BatchNormalization(), # Regularization
    layers.MaxPooling2D(), 
    
    layers.Conv2D(32, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2), # Regularization
    layers.Dense(9, activation="softmax"),
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(train_generator,validation_data=test_generator,epochs=50,shuffle=True)

Epoch 1/50
833/833 [==============================] - 464s 553ms/step - loss: 1.6507 - accuracy: 0.4166 - val_loss: 1.8904 - val_accuracy: 0.3194
Epoch 2/50
833/833 [==============================] - 459s 550ms/step - loss: 1.4190 - accuracy: 0.4913 - val_loss: 1.8655 - val_accuracy: 0.3198
Epoch 3/50
833/833 [==============================] - 455s 546ms/step - loss: 1.3411 - accuracy: 0.5191 - val_loss: 1.8143 - val_accuracy: 0.3524
Epoch 4/50
833/833 [==============================] - 442s 531ms/step - loss: 1.2732 - accuracy: 0.5483 - val_loss: 1.7455 - val_accuracy: 0.4006
Epoch 5/50
833/833 [==============================] - 447s 536ms/step - loss: 1.2211 - accuracy: 0.5708 - val_loss: 1.7548 - val_accuracy: 0.3784
Epoch 6/50
833/833 [==============================] - 446s 536ms/step - loss: 1.1709 - accuracy: 0.5879 - val_loss: 1.6693 - val_accuracy: 0.4131
Epoch 7/50
833/833 [==============================] - 447s 536ms/step - loss: 1.1283 - accuracy: 0.6018 - val_loss: 1.8864 -

In [21]:
model.save('CNN_model.h5')